In [ ]:
ver = 1 # version running when checking
Checker = "Rona"
Check_reason = "Daily Checking" # Khi check dispute nên đổi Check_reason để output trả ra đúng reason
template_sheet = "template" # sheet name of final template sheet
purpose = 'daily' # hiện tại để 2 purpose là daily - check order high risk hàng ngày & dispute - check dispute khi cần thiết

In [ ]:
from datetime import datetime
Check_date = datetime.today().strftime('%Y-%m-%d')

import pandas as pd
import numpy as np
import os
import requests as re
from copy import deepcopy

from google.colab import drive, auth, files
drive.mount('/content/drive')
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dict_deci = \
{'Alert1': 'Alert1',
 'Alert2': 'Alert2',
 'Alert4': 'Alert4',
 'Alert5': 'Alert5',
 'S1': 'S1',
 'H3': 'H3',
 'H5': 'H5',
 'M10': 'M10',
 'H8': 'H8'}

In [ ]:
dict_next_action = \
{'Alert1': 'Alert DA fix',
 'Alert2': 'Hold to check merchant test orders/other reason',
 'Alert4': 'Hold to check fraud buyer',
 'Alert5': 'Hold to check fraud products',
 'S1': 'Cancel & refund',
 'H3': 'Cancel & refund',
 'H5': 'Hold to confirm with buyer',
 'M10': 'Hold to confirm with buyer',
 'H8': 'Hold order to review store'}

## *Get data*



In [ ]:
uploaded = files.upload()
key, value = list(uploaded.items())[0]
data = pd.read_csv(key)
pre_df = deepcopy(data)
pre_df.loc[:, pre_df.columns != 'merchant_profit'] = pre_df.loc[:, pre_df.columns != 'merchant_profit'].fillna("")

Saving template to check rule đơn - 3449670 (25).csv to template to check rule đơn - 3449670 (25).csv


In [ ]:
df = deepcopy(pre_df)

## *Change column type*

In [ ]:
df['merchant_profit'] = df['merchant_profit'].fillna('9999999999')

In [ ]:
#change data type:

# df["No. orders associated with this customer email per day"] = \
  # df["No. orders associated with this customer email per day"].replace('',0).astype(float)
df['no_order_customer_per_day'] = \
  df['no_order_customer_per_day'].replace('',0).astype(float)
# df['count_order_of_customer_email'] = \
  # df['count_order_of_customer_email'].replace('',0).astype(float)
# df["Shipping addresses are (x) km far from card's address"] = \
  # df["Shipping addresses are (x) km far from card's address"].replace('',0).astype(float)
df["distance_billing_to_shipment"] = \
  df["distance_billing_to_shipment"].replace('',0).astype(float)
df["shipping_distance"] = \
  df["shipping_distance"].replace('',0).astype(float)
df["billing_distance"] = \
  df["billing_distance"].replace('',0).astype(float)
df["fraud_score"] = df["fraud_score"].replace('',0).astype(float)
df["payment_with_credit_card"] = \
  df["payment_with_credit_card"].replace('',0).astype(float)
df["gmv"] = df["gmv"].replace('',0).astype(float)
df["merchant_profit"] = df["merchant_profit"].replace('9,999,999,999.00', 9999999999).astype(float)
df["profit_margin"] = df["merchant_profit"]/df["gmv"]
df["total_quantity"] = df["total_quantity"].replace('',0).astype(float)
df["line_item_count"] = df["line_item_count"].replace('',0).astype(float)
df["margin_sale_cost"] = df["margin_sale_cost"].replace('',0).astype(float)
# df["sale_cost_rate"] = df["sale_cost_rate"].replace('',0).astype(float)
df["payment_attempt"] = df["payment_attempt"].replace('',0).astype(float)
df["email_fraud_score"] = df["email_fraud_score"].replace('',0).astype(float)
# df["Customer email AI score"] = df["Customer email AI score"].replace('',0).astype(float)
# df["count_dispute_same_ip"] = df["count_dispute_same_ip"].replace('',0).astype(float)
df["buyer_email"] = df["buyer_email"].str.lower()
df['count_order_of_customer_email'] = df["count_order_of_customer_email"].replace('',0).astype(float)
df['count_dispute_same_ip'] = df["count_dispute_same_ip"].replace('',0).astype(float)
df["stripe_risk_level"]=df["stripe_risk_level"].str.lower()
df['count_buyer_email_same_name'] = df["count_buyer_email_same_name"].replace('',0).astype(float)
df['count_buyer_name_same_email'] = df["count_buyer_name_same_email"].replace('',0).astype(float)
df['count_pmt_billing_addres_of_buyer'] = df["count_pmt_billing_addres_of_buyer"].replace('',0).astype(float)
df['no_customer_shop_per_merchant'] = df["no_customer_shop_per_merchant"].replace('',0).astype(float)

In [ ]:
if purpose == 'dispute':
  df['same_dispute_ip'] = ''
  df['count_dispute_same_ip'] = 0
else:
  df = df

In [ ]:
df.loc[df['merchant_profit'] == 9999999999, 'profit_margin'] = 0

## *Assign Significant*

In [ ]:
#significant
signal30S = df["pmt_decline_codes_reason"].str.contains("blocklist|elevated_risk_level|highest_risk_level|lost_card|pickup_card|restricted_card|security_violation|service_not_allowed|stolen_card|fraudulent")
signal38S = df["payment_attempt"] >= 15
signal42S = df["radar_early_fraud_warning"] != ""
signal43S = df["stripe_risk_level"].str.contains("highest")

## *Assign High*

In [ ]:
#High

# Bỏ signal 1H, 2H, 3H
# signal1H = df['is_valid_email']=="f"
# signal2H = df['is_disposable_email']=="t"
# signal3H = df["email_fraud_score"]==100
signal5H = df['buyer_email_match_user_email'] == 'match'
signal6H = df['count_buyer_email_same_name'] > 2
signal7H = df['count_buyer_name_same_email'] > 2
signal8H = df['count_pmt_billing_addres_of_buyer'] >= 3
signal9H = df['no_customer_shop_per_merchant'] >= 3
signal10H = df['buyer_match_this_merchant_ip_history'] == 'match'
signal11H = df['no_order_customer_per_day']>2
signal12H = df['count_order_of_customer_email']>2
signal13H = df['cvv']=="Card Verification Value (CVV) is incorrect"
signal15H = df["shipping_match_ip_country"]=="not match"
signal16H = ~((df["shipping_match_card_country"] .str.contains("Shipping country matches the card country|TRUE", na=False)) | \
              (df["shipping_match_card_country"]==""))
signal17H = df["billing_match_card_country"] =="Billing address ZIP/postal code doesn't match with credit card's registered address"
signal18H = ~((df["ip_match_card_country"] == "The IP country matches the card country") | \
              (df["ip_match_card_country"] == ""))
signal19H = df["billing_shipping_match"] == "not match"
signal20H = df["distance_billing_to_shipment"] > 1000
signal21H = df["shipping_distance"] > 1000
signal22H = df["billing_distance"] > 1000
signal23H = ~((df["billing_country"] == "Billing country matches the country from which the order was placed") | \
              (df["billing_country"]==""))
signal24H = df["ip_proxy"]=="Customer used a web proxy to fake their IP address"
signal25H = (df["same_dispute_ip"] != "This IP address doesn't match with any previous disputed IP") & \
            (df["same_dispute_ip"] != "This IP address isn't available to match") & \
            (df["same_dispute_ip"] != "")
signal26H = df["fraud_score"] > 75
signal27H = df["count_dispute_same_ip"] >= 3
signal28H = df["is_top_10_order_of_store"] == "yes"
signal29H = df["payment_with_credit_card"]>3
signal30H = df["pmt_decline_codes_reason"].str.contains("do_not_honor|transaction_not_allowed|incorrect_cvc", \
                                                        na = False)
signal31H = df["order_traffic_source"] == "direct"
signal32H = df["gmv"] >= 1000
signal33H = df["merchant_profit"] == 0
signal34H = df['profit_margin'] >= 0.8
signal35H = df["total_quantity"] >= 15
signal36H = df["line_item_count"] > 4
signal37H = df["margin_sale_cost"] < 0
signal38H = (df["payment_attempt"] >= 5) & (df["payment_attempt"] < 15)
signal39H = df["cvc_check"] == "fail"
signal40H = df["address_line1_check"] == "fail"
signal41H = df["address_postal_code_check"] == "fail"
signal43H = (~df["stripe_risk_level"].str.contains("highest", na=False)) & \
            (df["stripe_risk_level"].str.contains("elevated", na=False))
signal44H = df['payment_intent_card_funding'] == 'prepaid'

## *Assign Medium*

In [ ]:
# Medium

# Bỏ signal 3M
# signal3M = (df['email_fraud_score'] > 75) & (df['email_fraud_score'] <= 99)
signal4M = df['buyer_name_email_similar'] == 'buyer email not contain buyer name'
signal6M = df['count_buyer_email_same_name'] == 2
signal7M = df["count_buyer_name_same_email"] == 2
signal9M = df['no_customer_shop_per_merchant'] == 2
signal11M = df['no_order_customer_per_day'] == 2
signal12M = df['count_order_of_customer_email'] == 2
signal13M = df['cvv'] == "Card Verification Value (CVV) is unavailable"
signal14M = df['is_3d_secured_request'] == "3D Secure is NOT required for card"
signal17M = df["billing_match_card_country"] =="Billing address ZIP/postal code or credit card's registered address isn't available to match"
signal20M = (df['distance_billing_to_shipment'] > 150) & (df['distance_billing_to_shipment'] <= 1000)
signal21M = (df['shipping_distance'] > 150) & (df['shipping_distance'] <= 1000)
signal22M = (df['billing_distance'] > 150) & (df['billing_distance'] <= 1000)
signal25M = df["same_dispute_ip"] == "This IP address isn't available to match"
signal26M = (df['fraud_score'] >= 50) & (df['fraud_score'] <= 75)
signal27M = (df['count_dispute_same_ip'] >= 1) & (df['count_dispute_same_ip'] <= 2)
signal29M = (df['payment_with_credit_card'] >= 2) & (df['payment_with_credit_card'] <= 3)
signal30M = df["pmt_decline_codes_reason"].\
           str.contains("invalid_account|incorrect_number|requested_block_on_incorrect_cvc", na = False)
signal32M = (df['gmv'] >= 500) & (df['gmv'] < 1000)
signal34M = (df['profit_margin'] >= 0.5) & (df['profit_margin'] < 0.8)
signal35M = (df["total_quantity"] >= 5) & (df["total_quantity"] < 15)
signal36M = (df["line_item_count"] >= 3) & (df["line_item_count"] <= 4)
signal38M = df["payment_attempt"] == 2
signal39M = df["cvc_check"] == "unavailable"
signal40M = df["address_line1_check"] == "unavailable"
signal41M = df["address_postal_code_check"] == "unavailable"
signal43M = (~df["stripe_risk_level"].str.contains("highest", na=False)) & (~df["stripe_risk_level"].str.contains("elevated", na=False)) & (df["stripe_risk_level"].str.contains("unknown|not_assessed", na=False))
signal44M = df['payment_intent_card_funding'] == 'credit'

In [ ]:
# Regenarate significant signal stt
df['significant signal stt'] = ''

def get_significant_signal_stt(signal, signal_stt):
  df.loc[signal, 'significant signal stt'] = df.loc[signal, 'significant signal stt'] + str(signal_stt) + ' '

In [ ]:
list_sigficant_signal = [signal30S, signal38S, signal42S, signal43S]
list_significant_signal_stt = [30, 38, 42, 43]
for i in range(len(list_sigficant_signal)):
  get_significant_signal_stt(list_sigficant_signal[i], list_significant_signal_stt[i])

In [ ]:
# Regenarate high signal stt
df['high signal stt'] = ''

def get_high_signal_stt(signal, signal_stt):
    df.loc[signal, 'high signal stt'] = df.loc[signal, 'high signal stt'] + str(signal_stt) + ' '

In [ ]:
# Bỏ signal 1H, 2H, 3H
list_high_signal = [signal5H, signal6H, signal7H, signal8H, signal9H,
                    signal10H, signal11H, signal12H, signal13H, signal15H, signal16H, signal17H, signal18H, signal19H,
                    signal20H, signal21H, signal22H, signal23H, signal24H, signal25H, signal26H, signal27H, signal28H, signal29H,
                    signal30H, signal31H, signal32H, signal33H, signal34H, signal35H, signal36H, signal37H, signal38H, signal39H,
                    signal40H, signal41H, signal43H, signal44H]
list_high_signal_stt = [5, 6, 7, 8, 9,
                        10, 11, 12, 13, 15, 16, 17, 18, 19,
                        20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
                        30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
                        40, 41, 43, 44]
for i in range(len(list_high_signal)):
  get_high_signal_stt(list_high_signal[i], list_high_signal_stt[i])

In [ ]:
# Regenarate medium signal stt
df['medium signal stt'] = ''

def get_medium_signal_stt(signal, signal_stt):
    df.loc[signal, 'medium signal stt'] = df.loc[signal, 'medium signal stt'] + str(signal_stt) + ' '

In [ ]:
# Bỏ signal 3M
list_medium_signal = [signal4M, signal6M, signal7M, signal9M,
                      signal11M, signal12M, signal13M, signal14M, signal17M,
                      signal20M, signal21M, signal22M, signal25M, signal26M, signal27M, signal29M,
                      signal30M, signal32M, signal34M, signal35M, signal36M, signal38M, signal39M,
                      signal40M, signal41M, signal43M, signal44M]
list_medium_signal_stt = [4, 6, 7, 9,
                          11, 12, 13, 14, 17,
                          20, 21, 22, 25, 26, 27, 29,
                          30, 32, 34, 35, 36, 38, 39,
                          40, 41, 43, 44]
for i in range(len(list_medium_signal)):
  get_medium_signal_stt(list_medium_signal[i], list_medium_signal_stt[i])

### *Count signal*

In [ ]:
df['count_significant_risk'] = signal30S * 1 + signal38S * 1 + signal42S * 1 + signal43S * 1

In [ ]:
df['count_high_risk'] = signal5H * 1 + signal6H * 1 + signal7H * 1 + signal8H * 1 + \
signal9H * 1 + signal10H * 1 + signal11H * 1 + signal12H * 1 + signal13H * 1 + signal15H * 1 + signal16H * 1 + signal17H * 1 + signal18H * 1 + \
signal19H * 1 + signal20H * 1 + signal21H * 1 + signal22H * 1 + signal23H * 1 + signal24H * 1 + signal25H * 1 + signal26H * 1 + signal27H * 1 + \
signal28H * 1 + signal29H * 1 + signal30H * 1 + signal31H * 1 + signal32H * 1 + signal33H * 1 + signal34H * 1 + signal35H * 1 + signal36H * 1 + \
signal37H * 1 + signal38H * 1 + signal39H * 1 + signal40H * 1 + signal41H * 1 + signal43H * 1 + signal44H * 1

In [ ]:
df['count_medium_risk'] = signal4M * 1 + signal6M * 1 + signal7M * 1 + signal9M * 1 + signal11M * 1 + signal12M * 1 + \
signal13M * 1 + signal14M * 1 + signal17M * 1 + signal20M * 1 + signal21M * 1 + signal22M * 1 + signal25M * 1 + signal26M * 1 + signal27M * 1 + \
signal29M * 1 + signal30M * 1 + signal32M * 1 + signal34M * 1 + signal35M * 1 + signal36M * 1 + signal38M * 1 + signal39M * 1 + \
signal40M * 1 + signal41M * 1 + signal43M * 1 + signal44M * 1

In [ ]:
pd.options.mode.chained_assignment = None

## *Assign continuing*

In [ ]:
# Alert1
Alert1 = df['merchant_profit'] == 9999999999

# Alert2
Alert2 = signal5H | signal33H

# Alert4
Alert4 = df['count_order_of_customer_email'] >= 3

# Alert5
Alert5 = signal34H

# S1
S1 = signal30S | signal38S | signal42S | signal43S
#########################

# H3
H3 = (df['count_high_risk'] > 8) & (~signal28H)

# H9:
# >= 2H
H9 = (df['count_high_risk'] >= 2) & ((signal13H & signal38H) | (signal34H))

# H5
# 1H_signal
H5 = (df['count_high_risk'] == 1) & \
     (signal11H | signal23H | signal30H | signal32H | signal34H | signal35H | signal38H | signal39H)

# M10
M10 = df['count_medium_risk'] >= 10

# H8
H8 = signal28H

## *Assign Signals rules*

In [ ]:
df['Signals rules'] = ""

# S1
df.loc[S1, "Signals rules"] =dict_deci["S1"]
exclude = df["Signals rules"] != dict_deci['S1']

# H3
df.loc[H3 & exclude, "Signals rules"] =dict_deci["H3"]
exclude = exclude & (df["Signals rules"] != dict_deci['H3'])

# Alert1
df.loc[Alert1 & exclude, "Signals rules"] = dict_deci["Alert1"]
exclude = exclude & (df["Signals rules"] != dict_deci['Alert1'])

# Alert2
df.loc[Alert2 & exclude, "Signals rules"] =dict_deci["Alert2"]
exclude = exclude & (df["Signals rules"] != dict_deci['Alert2'])

# Alert5
df.loc[Alert5 & exclude, "Signals rules"] = dict_deci["Alert5"]
exclude = exclude & (df["Signals rules"] != dict_deci['Alert5'])

# Alert4
df.loc[Alert4 & exclude, "Signals rules"] = dict_deci["Alert4"]
exclude = exclude & (df["Signals rules"] != dict_deci['Alert4'])

# H8
df.loc[H8 & exclude, "Signals rules"] = dict_deci["H8"]
exclude = exclude & (df["Signals rules"] != dict_deci['H8'])

# H5
df.loc[H5 & exclude, "Signals rules"] = dict_deci["H5"]
exclude = exclude & (df["Signals rules"] != dict_deci['H5'])

# M10
df.loc[M10 & exclude, "Signals rules"] =dict_deci["M10"]
exclude = exclude & (df["Signals rules"] != dict_deci['M10'])

## *Assign Ref rule*

In [ ]:
df['Ref_rule'] = ""

# S1
df.loc[S1, "Ref_rule"] = df["significant signal stt"] + str('S')
exclude = df["Ref_rule"] != df["significant signal stt"] + str('S')

# H3
df.loc[H3 & exclude, "Ref_rule"] = df['high signal stt'] + str('H')
exclude = exclude & (df["Ref_rule"] != df['high signal stt'] + str('H'))

# Alert1
df.loc[Alert1 & exclude, "Ref_rule"] = dict_deci["Alert1"]
exclude = exclude & (df["Ref_rule"] != dict_deci['Alert1'])

# Alert2
df.loc[Alert2 & exclude, "Ref_rule"] = dict_deci["Alert2"]
exclude = exclude & (df["Ref_rule"] != dict_deci['Alert2'])

# Alert5
df.loc[Alert5 & exclude, "Ref_rule"] = dict_deci["Alert5"]
exclude = exclude & (df["Ref_rule"] != dict_deci['Alert5'])

# Alert4
df.loc[Alert4 & exclude, "Ref_rule"] = dict_deci["Alert4"]
exclude = exclude & (df["Ref_rule"] != dict_deci['Alert4'])

# H8
df.loc[H8 & exclude, "Ref_rule"] = df['high signal stt'] + str('H')
exclude = exclude & (df["Ref_rule"] != df['high signal stt'] + str('H'))

# H5
df.loc[H5 & exclude, "Ref_rule"] = df['high signal stt'] + str('H')
exclude = exclude & (df["Ref_rule"] != df['high signal stt'] + str('H'))

# M10
df.loc[M10 & exclude, "Ref_rule"] = df['medium signal stt'] + str('M')
exclude = exclude & (df["Ref_rule"] != df['medium signal stt'] + str('M'))

## *Assign Next actions*

In [ ]:
#next action:

df["Next_action"] = "Release to fulfill"
for k in dict_deci:
  # print(k)
  df.loc[df["Signals rules"]==dict_deci[k], "Next_action"] = dict_next_action[k]

### *Test check signals rules*

In [ ]:
#count test
df.groupby("Next_action").count()

,shop_id,order_pl_type,order_id,order_name,order_created_at,link_hive,buyer_email,payment_gateway,buyer_name_email_similar,no_order_customer_per_day,...,user_id,user_email,significant signal stt,high signal stt,medium signal stt,count_significant_risk,count_high_risk,count_medium_risk,Signals rules,Ref_rule
Next_action,,,,,,,,,,,,,,,,,,,,,
Release to fulfill,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [ ]:
df_count = df.groupby("Signals rules").count()
df_count.sort_index()

,shop_id,order_pl_type,order_id,order_name,order_created_at,link_hive,buyer_email,payment_gateway,buyer_name_email_similar,no_order_customer_per_day,...,user_id,user_email,significant signal stt,high signal stt,medium signal stt,count_significant_risk,count_high_risk,count_medium_risk,Ref_rule,Next_action
Signals rules,,,,,,,,,,,,,,,,,,,,,
,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [ ]:
df.loc[df['merchant_profit'] == 9999999999, 'merchant_profit'] = ''

In [ ]:
df.shape[0]

3

In [ ]:
worksheet = gc.open(template_sheet).sheet1
rows = worksheet.get_all_values()
template = pd.DataFrame.from_records(rows)
template = template.rename(columns=template.iloc[0]).drop(template.index[0])
df_final = deepcopy(template)

In [ ]:
df["Checker"] = Checker
df["Check_date"] = Check_date
df["Check_reason"] = Check_reason
for i in range(len(df.columns)):
  df_final[df.columns[i]] = df.iloc[:,i]
df_final = df_final.fillna("")
order_type = df_final.loc[0, 'order_pl_type']

In [ ]:
from gspread_dataframe import set_with_dataframe

final_gsheet = "{}_output_{}_{}_{}".format(order_type, Checker, Check_date, ver)
gc.create(final_gsheet)
sheet = gc.open(final_gsheet).sheet1
set_with_dataframe(sheet, df_final)

In [ ]:
# df_final.to_csv("{}_output_{}_{}_{}.csv".format(order_type, Checker, Check_date, ver))
# files.download("{}_output_{}_{}_{}.csv".format(order_type, Checker, Check_date, ver))

### *Write hold data to sheet summary*

In [ ]:
def summary_plb(key):
  ws = gc.open_by_key(key)
  summary_sheet = ws.worksheet('Summary')
  summary_data = pd.DataFrame.from_records(summary_sheet.get_all_records(head=1)) # Đọc dữ liệu.
  summary_header = summary_data.iloc[0:1, :]
  summary_data = summary_data.loc[1:, :]
  df_summary = df_final[['Check_reason', 'user_id', 'user_email', 'shop_id', 'order_name', 'order_id', 'order_created_at', 'link_hive', 'buyer_email', 'payment_gateway', 'Final_action', 'Signals rules', 'Ref_rule', 'Next_action']]
  df_summary = df_summary[df_summary['Next_action'] != 'Release to fulfill'] # Lấy các orders có Next_action != 'Release to fulfill'
  df_summary.rename(columns={'Check_reason':'check_reason',
                            'user_id':'user_id',
                            'user_email':'user_email',
                            'shop_id':'shop_id',
                            'order_name':'order_name',
                            'order_id':'order_id',
                            'order_created_at':'create_at',
                            'link_hive':'link_hive',
                            'buyer_email':'buyer_email',
                            'payment_gateway':'payment_gateways',
                            'Final_action':'final_action',
                            'Signals rules':'signals_rules',
                            'Ref_rule':'ref_rule',
                            'Next_action':'next_action'}, inplace=True)
  df_summary['create_at'] = pd.to_datetime(df_summary['create_at'], utc=True)
  df_summary['label'] = ''
  df_summary['week_id'] = df_summary['create_at'].dt.isocalendar().year.apply(str) + df_summary['create_at'].dt.isocalendar().week.apply(str)
  df_summary = df_summary[summary_data.columns].sort_values('create_at', ascending=False)
  df_summary['create_at'] = df_summary['create_at'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
  summary_daily = pd.concat([summary_header, df_summary, summary_data])
  summary_daily.fillna('', inplace=True)
  summary_sheet.clear()
  summary_sheet.update('A1:P',[summary_daily.columns.to_list()]+ summary_daily.to_numpy().tolist()) # Ghi data lên google sheet "Summary"

In [ ]:
if order_type == 'PLB':
  summary_plb('1A8Jy9P2XpP5YE0gxxJMq6iIEZMADukx3YvJQMTmMjuw')
elif order_type == 'PB':
  summary_plb('1n3u9JH8DsMoI9jBD7NAfwBkeOrMvv97eQs9ZAVCFLRM')